## 非线性 SVM 例子
---

本节讲述如何使用 tensorflow 在 iris 数据集上实现高斯内核。

高斯内核：

$$K(x_1, x_2) = \exp \left(-\gamma * (x_1 - x_2)^2 \right)$$

In [1]:
# load necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [2]:
# Create a graph session
sess = tf.Session()

### 加载 iris 数据

我们的 x 值将是 (x1, x2)，其中 x1 = 'Sepal Length'，x2 = 'Petal Width'。

目标值将是花的种类是否是 Setosa。

In [3]:
# Load the data
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals = np.array([1 if y==0 else -1 for y in iris.target])
class1_x = [x[0] for i,x in enumerate(x_vals) if y_vals[i]==1]
class1_y = [x[1] for i,x in enumerate(x_vals) if y_vals[i]==1]
class2_x = [x[0] for i,x in enumerate(x_vals) if y_vals[i]==-1]
class2_y = [x[1] for i,x in enumerate(x_vals) if y_vals[i]==-1]

### 模型参数

In [4]:
# Declare batch size
batch_size = 150

# Initialize placeholders
x_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
prediction_grid = tf.placeholder(shape=[None, 2], dtype=tf.float32)

# Create variables for svm
b = tf.Variable(tf.random_normal(shape=[1,batch_size]))

### 高斯（RBF）内核

我们创建高斯内核用来转换数据点集到高维空间。

两点内核，$x$ 和 $x'$ 如下：

$$K(x, x') = \exp \left(-\gamma \|x - x'\|^2 \right)$$

$\gamma$ 越小，内核越宽，反之亦然。这意味着大的 $\gamma$ 导致